# **Data Wrangling**


## Objectives


-   Identify duplicate values in the dataset.

-   Remove duplicate values from the dataset.

-   Identify missing values in the dataset.

-   Impute the missing values in the dataset.

-   Normalize data in the dataset.


<hr>


Import pandas module.


In [20]:
import pandas as pd

Load the dataset into a dataframe.


<h2>Read Data</h2>

In [2]:
from pyodide.http import pyfetch

async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, "wb") as f:
            f.write(await response.bytes())

In [3]:
file_path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/LargeData/m1_survey_data.csv"

To obtain the dataset, utilize the download() function as defined above:  


In [4]:
await download(file_path, "m1_survey_data.csv")
file_name="m1_survey_data.csv"

Utilize the Pandas method read_csv() to load the data into a dataframe.


In [5]:
df = pd.read_csv(file_name)

## Finding duplicates


 Find how many duplicate rows exist in the dataframe.


In [7]:
num_duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 154


## Removing duplicates


Remove the duplicate rows from the dataframe.


In [8]:
df = df.drop_duplicates()

Verify if duplicates were actually dropped.


In [9]:
num_duplicates_after = df.duplicated().sum()
print(f"Number of duplicate rows after dropping: {num_duplicates_after}")

Number of duplicate rows after dropping: 0


## Finding Missing values


Find the missing values for all columns.


In [10]:
missing_values = df.isnull().sum()
print("Missing values for all columns:")
print(missing_values)

Missing values for all columns:
Respondent        0
MainBranch        0
Hobbyist          0
OpenSourcer       0
OpenSource       81
               ... 
Sexuality       542
Ethnicity       675
Dependents      140
SurveyLength     19
SurveyEase       14
Length: 85, dtype: int64


Find out how many rows are missing in the column 'WorkLoc'


In [11]:
missing_workloc = df['WorkLoc'].isnull().sum()
print(f"Number of missing rows in 'WorkLoc': {missing_workloc}")

Number of missing rows in 'WorkLoc': 32


## Imputing missing values


Find the  value counts for the column WorkLoc.


In [12]:
workloc_value_counts = df['WorkLoc'].value_counts()
print("Value counts for 'WorkLoc':")
print(workloc_value_counts)

Value counts for 'WorkLoc':
WorkLoc
Office                                            6806
Home                                              3589
Other place, such as a coworking space or cafe     971
Name: count, dtype: int64


Identify the value that is most frequent (majority) in the WorkLoc column.


In [13]:
majority_value = workloc_value_counts.idxmax()
print(f"Most frequent value in 'WorkLoc': {majority_value}")

Most frequent value in 'WorkLoc': Office


Impute (replace) all the empty rows in the column WorkLoc with the value that you have identified as majority.


In [15]:
df['WorkLoc'].fillna(majority_value, inplace=True)

After imputation there should ideally not be any empty rows in the WorkLoc column.


Verify if imputing was successful.


In [16]:
missing_workloc_after = df['WorkLoc'].isnull().sum()
print(f"Number of missing rows in 'WorkLoc' after imputation: {missing_workloc_after}")

Number of missing rows in 'WorkLoc' after imputation: 0


## Normalizing data


There are two columns in the dataset that talk about compensation.

One is "CompFreq". This column shows how often a developer is paid (Yearly, Monthly, Weekly).

The other is "CompTotal". This column talks about how much the developer is paid per Year, Month, or Week depending upon his/her "CompFreq". 

This makes it difficult to compare the total compensation of the developers.

Create a new column called 'NormalizedAnnualCompensation' which contains the 'Annual Compensation' irrespective of the 'CompFreq'.

Once this column is ready, it makes comparison of salaries easy.

<hr>


List out the various categories in the column 'CompFreq'


In [17]:
compfreq_categories = df['CompFreq'].unique()
print("Categories in 'CompFreq':")
print(compfreq_categories)

Categories in 'CompFreq':
['Yearly' 'Monthly' 'Weekly' nan]


Create a new column named 'NormalizedAnnualCompensation'.


In [19]:
def normalize_annual_compensation(row):
    if row['CompFreq'] == 'Yearly':
        return row['CompTotal']
    elif row['CompFreq'] == 'Monthly':
        return row['CompTotal'] * 12
    elif row['CompFreq'] == 'Weekly':
        return row['CompTotal'] * 52
    else:
        return None

df['NormalizedAnnualCompensation'] = df.apply(normalize_annual_compensation, axis=1)

print(df[['CompFreq', 'CompTotal', 'NormalizedAnnualCompensation']].head())

  CompFreq  CompTotal  NormalizedAnnualCompensation
0   Yearly    61000.0                       61000.0
1   Yearly   138000.0                      138000.0
2   Yearly    90000.0                       90000.0
3  Monthly    29000.0                      348000.0
4   Yearly    90000.0                       90000.0


<!--| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-10-17        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |--!>
